In [ ]:
import h5py
import pandas as pd
import tensorflow
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
#from matplotlib import pyplot
import matplotlib.pyplot as plt
#import signal
import random
#import h5py
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
# load vgg model
#from keras.applications.vgg16 import VGG16
# load the model
#model = VGG16()
# summarize the model
#model.summary()

GCD=math.gcd
X,Y=0,0
batch_size = 25
loss_function = 'categorical_crossentropy'
no_classes = 60
no_epochs = 25
optimizer = Adam()
validation_split = 0.5
verbosity = 1

f = h5py.File('/content/DB_AerosalIndex.he5', 'r')
label_train = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['UVAerosolIndex'][...]
input_train = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['CloudFraction'][...]
print(input_train.shape)
#print(input_test.shape)
f.close()
f = h5py.File('/content/DB_AerosalIndex_testing.he5', 'r')
input_test = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['CloudFraction'][...]
label_test = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['UVAerosolIndex'][...]
f.close()
#print(input_train)
A = len(input_train)
B = len(input_test)
X = GCD(len(input_train), len(input_test))
Y = input_train.shape[0]*input_train.shape[1]
img_width, img_num_channels = 60, 1
input_train = input_train.reshape((X, img_width, int(A/X), img_num_channels))
input_test  = input_test.reshape((X, img_width, int(B/X), img_num_channels))
#print(len(input_test))

#input_train = input_train.reshape((len(input_train), img_width, img_height, img_num_channels))
#input_test  = input_test.reshape((len(input_test), img_width, img_height, img_num_channels))

#input_train = np.resize(input_train, (50,50))
#input_test  = np.resize(input_test, (50,50))
input_train = np.asarray(input_train)
input_test = np.asarray(input_test)

input_shape = (img_width, int(A/X) , img_num_channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='sigmoid'))

model.summary()

for layer in model.layers:
  if 'conv' not in layer.name:
    continue
  else:
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

filters, biases = model.layers[1].get_weights()
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
n_filters, ix = 6, 1
for i in range(n_filters):
  f = filters[:, :, :, i]
  for j in range(3):
    ax = plt.subplot(n_filters, 3, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(f[:, :, j], cmap='gray')
    ix += 1
plt.show()

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])
print('loss')

history = model.fit(input_train, label_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split = validation_split)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(X_train[10].reshape(1,28,28,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

score = model.evaluate(input_test, label_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


In [ ]:
import h5py
import pandas as pd
import tensorflow
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
#import signal
import random
#import h5py
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

GCD=math.gcd
X,Y=0,0
batch_size = 25
loss_function = 'categorical_crossentropy'
no_classes = 60
no_epochs = 25
optimizer = Adam()
validation_split = 0.2
verbosity = 1

f = h5py.File('DB_AerosalIndex.he5', 'r')
label_train = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['UVAerosolIndex'][...]
input_train = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['CloudOpticalDepth'][...]
print(input_train.shape)
#print(input_test.shape)
f.close()
f = h5py.File('DB_AerosalIndex_testing.he5', 'r')
input_test = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['CloudOpticalDepth'][...]
label_test = f['HDFEOS']['SWATHS']['Aerosol NearUV Swath']['Data Fields']['UVAerosolIndex'][...]
f.close()
#print(input_train)
A = len(input_train)
B = len(input_test)
X = GCD(len(input_train), len(input_test))
Y = input_train.shape[0]*input_train.shape[1]
img_width, img_num_channels = 60, 1
input_train = input_train.resize((X, img_width, int(A/X), img_num_channels))
input_test  = input_test.resize((X, img_width, int(B/X), img_num_channels))
#print(len(input_test))

#input_train = input_train.reshape((len(input_train), img_width, img_height, img_num_channels))
#input_test  = input_test.reshape((len(input_test), img_width, img_height, img_num_channels))


#input_train = np.resize(input_train, (50,50))
#input_test  = np.resize(input_test, (50,50))

input_shape = (img_width, int(A/X) , img_num_channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='sigmoid'))

model.summary()

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])
print('loss')

history = model.fit(input_train, label_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)

score = model.evaluate(input_test, label_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
#from pyhdf.SD import SD, SDC

#file_name = 'MYD06_L2.A2007219.2010.006.2014053202546.hdf'
#file = SD(file_name, SDC.READ)

#print file.info()


(1701, 60)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 565, 32)       320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 563, 64)       18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 54, 561, 128)      73856     
_________________________________________________________________
flatten (Flatten)            (None, 3877632)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               496337024 
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7740      
Total params: 496,437,436
Trainable params: 496,437,436
Non-trainable params: 0
_______________________________

In [1]:
import h5py
filename = "DB_AerosalIndex.he5"
import os

#hdf_file = 'MYD08_M3.A2002182.051.2008334061251.psgscs_000500751197.hdf'

if not os.path.isfile(filename):
    print ('file %s not found' % filename)

if not os.access(filename, os.R_OK):
    print ('file %s not readable' % filename)
#print('ok')

f = h5py.File(filename, 'r')

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])[:]
    print(data)
    
    #data1 = list(f[data])[:]
    #print(data1)
    #swaths = list(f[data])[0]
    #print(swaths)
    

Keys: <KeysViewHDF5 ['HDFEOS', 'HDFEOS INFORMATION']>
['ADDITIONAL', 'SWATHS']


In [ ]:
import h5py
import pandas as pd
import tensorflow
import numpy as np
import math
#import h5py
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

GCD=math.gcd
X,Y=0,0
batch_size = 25
loss_function = 'categorical_crossentropy'
no_classes = 60
no_epochs = 25
optimizer = Adam()
validation_split = 0.2
verbosity = 1

f = h5py.File('DB_Sulfar.he5', 'r')
label_train = f['HDFEOS']['SWATHS']['CO']['Data Fields']['Quality'][...]
input_train = f['HDFEOS']['SWATHS']['CO']['Data Fields']['CO'][...]
print(input_train.shape)
#print(input_test.shape)
f.close()
f = h5py.File('DB_Sulfar_2.he5', 'r')
input_test = f['HDFEOS']['SWATHS']['CO']['Data Fields']['CO'][...]
label_test = f['HDFEOS']['SWATHS']['CO']['Data Fields']['Quality'][...]
f.close()
#print(input_train)
A = len(input_train)
B = len(input_test)
X = GCD(len(input_train), len(input_test))
Y = input_train.shape[0]*input_train.shape[1]
img_width, img_num_channels = 60, 1
input_train = input_train.resize((X, img_width, int(A/X), img_num_channels))
input_test  = input_test.resize((X, img_width, int(B/X), img_num_channels))
#print(len(input_test))

#input_train = input_train.reshape((len(input_train), img_width, img_height, img_num_channels))
#input_test  = input_test.reshape((len(input_test), img_width, img_height, img_num_channels))


#input_train = np.resize(input_train, (50,50))
#input_test  = np.resize(input_test, (50,50))

input_shape = (img_width, int(A/X) , img_num_channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='sigmoid'))

model.summary()

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])
print('loss')

history = model.fit(input_train, label_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)

score = model.evaluate(input_test, label_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
#from pyhdf.SD import SD, SDC

#file_name = 'MYD06_L2.A2007219.2010.006.2014053202546.hdf'
#file = SD(file_name, SDC.READ)

#print file.info()


In [2]:
    def visualize_model(model_path,model_weight_path, img_paths):
        model = load_model(model_path)
        model.load_weights(model_weight_path)

        successive_outputs = [layer.output for layer in model.layers[1:]]
        # visualization_model = Model(img_input, successive_outputs)
        visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs)
        #img_path = random.choice(img_paths)                    

        img = load_img(img_paths, target_size=(224, 224))  # this is a PIL image
        x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
        x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

        # Rescale by 1/255
        x /= 255.

        successive_feature_maps = visualization_model.predict(x)

        layer_names = [layer.name for layer in model.layers[1:]]

        for layer_name, feature_map in zip(layer_names, successive_feature_maps):
            if len(feature_map.shape) == 4:
                n_features = feature_map.shape[-1]
                # The feature map has shape (1, size, size, n_features)
                size = feature_map.shape[1]
                display_grid = np.zeros((size, size * n_features))
                for i in range(n_features):
                    x = feature_map[0, :, :, i]
                    x -= x.mean()
                    x /= x.std()
                    x *= 64
                    x += 128
                    x = np.clip(x, 0, 255).astype('uint8')
                    display_grid[:, i * size: (i + 1) * size] = x
                scale = 20. / n_features
                plt.figure(figsize=(scale * n_features, scale))
                plt.title(layer_name)
                plt.grid(False)
                plt.imshow(display_grid, aspect='auto', cmap='viridis')